In [1]:
#Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB


In [15]:
# # Load all CSV files
Demand_Data_Normalized = pd.read_csv('../Data\ModelData\Demand_YearlyDemandUtilityProfile_Normalized.csv', delimiter=',')
Fuel_Cost_Data_Normalized = pd.read_csv('../Data\ModelData\FuelCost_PriceDevelopment50years.csv', delimiter=',')
Generation_Data_Normalized = pd.read_csv('../Data\ModelData\VRE_YearlyGenerationProfile_Normalized.csv', delimiter=',')
Generation_Asset_Data_Existing = pd.read_csv('../Data\ModelData\Generators_AssetData_Existing.csv', delimiter=',')
Demand_Unit_Data = pd.read_csv('../Data\ModelData\Demand_UnitSpecificData.csv', delimiter=',')
System_Demand = pd.read_csv('../Data\ModelData\System_Demand.csv', delimiter=',')
Transmission_Capacity = pd.read_csv('../Data\ModelData\Transmission Capacity.csv', delimiter=',')

## Problem 1 - Market clearing as price taker where investment does not influence the market clearing

In [ ]:
#Creating a function for the market clearing

def MarketClearingProblem1(Demand_Data_Normalized,
                           Fuel_Cost_Data_Normalized,
                           Generation_Data_Normalized,
                           Generation_Asset_Data_Existing,
                           Demand_Unit_Data,
                           System_Demand,
                           Transmission_Capacity,
                           Season):
    model = gp.Model("MarketClearingProblem1")

    #Defining some iput data

    #Define the number of generators by taking the length of the Generation_Asset_Data_Existing file
    Number_of_Generators = len(Generation_Asset_Data_Existing)
    Number_of_Price_Zones = 2
    Number_of_Demand_Units = len(Demand_Unit_Data)
    Number_of_Hours = 24
    #-----------------------------------------------------------------------------------------------------------------

    #Data Handling

    # Filter Demand_Data_Normalized by the given season
    season_data = Demand_Data_Normalized[Demand_Data_Normalized['Season'] == Season]

    # Create a empty DataFrames to store the results, with 24 columns for each hour
    Daily_Demand_Normalized = pd.DataFrame(columns=[f'Hour_{i+1}' for i in range(24)])
    Daily_Demand_Utility = pd.DataFrame(columns=[f'Hour_{i+1}' for i in range(24)])

    # Loop through each row in Demand_Unit_Data
    for _, row in Demand_Unit_Data.iterrows():
        load_type = row['LoadType'].strip() # Load type (e.g., 'U_Residential', 'U_IndustryBase', 'U_IndustryPeak')
        utility_type = row['UtilityType'].strip()
        percent_load = row['% of system load'] / 100
        
        # Check if the load type exists in the Demand_Data_Normalized columns
        if load_type in season_data.columns:
            # Retrieve the hourly values for the specified load type and scale them
            hourly_values_loadtype = season_data[season_data['Hour'] <= 24][load_type].values * percent_load
            
            # Create a new row with these hourly values
            row_data = pd.DataFrame([hourly_values_loadtype], columns=[f'Hour_{i+1}' for i in range(24)])
            
            # Add identifying columns (like Load #, Zone, and LoadType) for clarity
            # row_data['Load #'] = row['Load #']
            # row_data['Zone'] = row['Zone']
            # row_data['LoadType'] = load_type
            
            # Append the new row to the result_data DataFrame
            Daily_Demand_Normalized = pd.concat([Daily_Demand_Normalized, row_data], ignore_index=True)
        # Check if the utility type exists in the Demand_Data_Normalized columns
        if utility_type in season_data.columns:
            # Retrieve the hourly values for the specified load type and scale them
            hourly_values_utilitytype = season_data[season_data['Hour'] <= 24][utility_type].values 
            
            # Create a new row with these hourly values
            row_data = pd.DataFrame([hourly_values_utilitytype], columns=[f'Hour_{i+1}' for i in range(24)])
            
            # Add identifying columns (like Load #, Zone, and LoadType) for clarity
            # row_data['Load #'] = row['Load #']
            # row_data['Zone'] = row['Zone']
            # row_data['LoadType'] = utility_type
            
            # Append the new row to the result_data DataFrame
            Daily_Demand_Utility = pd.concat([Daily_Demand_Utility, row_data], ignore_index=True)

    # Reset index and inspect result
    Daily_Demand_Normalized.reset_index(drop=True, inplace=True)
    Daily_Demand_Utility.reset_index(drop=True, inplace=True)


    #getting from normalized load to absolute load values
    Daily_Demand_Absolute = pd.DataFrame(columns=[f'Hour_{i+1}' for i in range(24)])
    for l in range(len(Daily_Demand_Normalized)):
         for h in range(len(System_Demand)):
            Daily_Demand_Absolute[l,h] = Daily_Demand_Normalized[l,h]*System_Demand['System demand (MW)'][h]



   

    #-----------------------------------------------------------------------------------------------------------------

    #Defining the decision variables

    # Defining a variable for the generators day ahead bid
    Gen_DABid = model.addVars(Number_of_Generators,Number_of_Hours, vtype=GRB.CONTINUOUS, name="Gen_DABid")
     # Defining a variable for the demand day ahead bid
    Dem_DABid = model.addVars(Number_of_Demand_Units,Number_of_Hours, vtype=GRB.CONTINUOUS, name="Dem_DABid")
    # Define voltage angle variables for each zone
    Voltage_Angle = model.addVars(Number_of_Price_Zones,Number_of_Hours vtype=GRB.CONTINUOUS, name="Voltage_Angle")
    # Define power flow variables (as before, but now dependent on voltage angles)
    Power_Flow = model.addVars(len(Transmission_Capacity),Number_of_Hours, vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, name="Power_Flow")
    
    #-----------------------------------------------------------------------------------------------------------------

    #Defining objective function

    #Objective function that maximizes social welfare
    objective = (gp.quicksum(Generation_Asset_Data_Existing['C_i ($/MWh)'][g] * Gen_DABid[g,h] for g in range(Number_of_Generators) for h in range(Number_of_Hours)) 
                - gp.quicksum(Daily_Demand_Utility[d,h] * Dem_DABid[d,h] for d in range(Number_of_Demand_Units) for h in range(Number_of_Hours)))
    model.setObjective(objective, GRB.MINIMIZE)
    
    # Add slack bus constraint: Fix the voltage angle of node 1 (or any other node) to 0 degrees
    model.addConstr(Voltage_Angle[0] == 0, name="Slack_Bus")




In [ ]:
df = pd.DataFrame(index=range(len(Demand_Unit_Data)), columns=[f"Hour{i}" for i in range(Number_of_Hours)])

In [ ]:


Season_filtered_Demand_Data = Demand_Data_Normalized[Demand_Data_Normalized['Season'] == season]

In [ ]:
Season_filtered_Demand_Data['LoadType']

In [ ]:
LoadType = 'D_residential'

In [ ]:


# Assume Demand_Unit_Data and Demand_Data_Normalized are already loaded DataFrames
# season_value is provided externally

# Filter Demand_Data_Normalized by the given season
season_data = Demand_Data_Normalized[Demand_Data_Normalized['Season'] == Season]

# Create an empty DataFrame to store the results, with 24 columns for each hour
Daily_Demand_Normalized = pd.DataFrame(columns=[f'Hour_{i+1}' for i in range(24)])
Daily_Demand_Utility_Normalized = pd.DataFrame(columns=[f'Hour_{i+1}' for i in range(24)])

# Loop through each row in Demand_Unit_Data
for _, row in Demand_Unit_Data.iterrows():
    load_type = row['LoadType'].strip() # Load type (e.g., 'U_Residential', 'U_IndustryBase', 'U_IndustryPeak')
    utility_type = row['UtilityType'].strip()
    percent_load = row['% of system load'] / 100  # Scale factor as a fraction
    
    # Check if the load type exists in the Demand_Data_Normalized columns
    if load_type in season_data.columns:
        # Retrieve the hourly values for the specified load type and scale them
        hourly_values_loadtype = season_data[season_data['Hour'] <= 24][load_type].values 
        
        # Create a new row with these hourly values
        row_data = pd.DataFrame([hourly_values_loadtype], columns=[f'Hour_{i+1}' for i in range(24)])
        
        # Add identifying columns (like Load #, Zone, and LoadType) for clarity
        row_data['Load #'] = row['Load #']
        row_data['Zone'] = row['Zone']
        row_data['LoadType'] = load_type
        
        # Append the new row to the result_data DataFrame
        Daily_Demand_Normalized = pd.concat([Daily_Demand_Normalized, row_data], ignore_index=True)
    # Check if the utility type exists in the Demand_Data_Normalized columns
    if utility_type in season_data.columns:
        # Retrieve the hourly values for the specified load type and scale them
        hourly_values_utilitytype = season_data[season_data['Hour'] <= 24][utility_type].values 
        
        # Create a new row with these hourly values
        row_data = pd.DataFrame([hourly_values_utilitytype], columns=[f'Hour_{i+1}' for i in range(24)])
        
        # Add identifying columns (like Load #, Zone, and LoadType) for clarity
        row_data['Load #'] = row['Load #']
        row_data['Zone'] = row['Zone']
        row_data['LoadType'] = utility_type
        
        # Append the new row to the result_data DataFrame
        Daily_Demand_Utility_Normalized = pd.concat([Daily_Demand_Utility_Normalized, row_data], ignore_index=True)

# Reset index and inspect result
Daily_Demand_Normalized.reset_index(drop=True, inplace=True)
Daily_Demand_Utility_Normalized.reset_index(drop=True, inplace=True)
print(Daily_Demand_Normalized)
print(Daily_Demand_Utility_Normalized)

      Hour_1    Hour_2    Hour_3    Hour_4    Hour_5    Hour_6    Hour_7  \
0   0.734934  0.716599  0.716209  0.731226  0.759359  0.757673  0.758828   
1   1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
2   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.300000   
3   0.734934  0.716599  0.716209  0.731226  0.759359  0.757673  0.758828   
4   1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
5   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.300000   
6   0.734934  0.716599  0.716209  0.731226  0.759359  0.757673  0.758828   
7   1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
8   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.300000   
9   0.734934  0.716599  0.716209  0.731226  0.759359  0.757673  0.758828   
10  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
11  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.300000   
12  0.734934

C:\Users\konst\AppData\Local\Temp\ipykernel_36360\1881609469.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Daily_Demand_Normalized = pd.concat([Daily_Demand_Normalized, row_data], ignore_index=True)


In [3]:
Season = 1

In [30]:
# #Creating a function for the market clearing


model = gp.Model("MarketClearingProblem1")

# #Defining some iput data

# #Define the number of generators by taking the length of the Generation_Asset_Data_Existing file
Number_of_Generators = len(Generation_Asset_Data_Existing)
Number_of_Price_Zones = 2
Number_of_Demand_Units = len(Demand_Unit_Data)
Number_of_Hours = 24
# #-----------------------------------------------------------------------------------------------------------------

# #Data Handling

# Filter Demand_Data_Normalized by the given season
season_data = Demand_Data_Normalized[Demand_Data_Normalized['Season'] == Season]

# Create a empty DataFrames to store the results, with 24 columns for each hour
Daily_Demand_Normalized = pd.DataFrame(columns=[f'Hour_{i+1}' for i in range(24)])
Daily_Demand_Utility = pd.DataFrame(columns=[f'Hour_{i+1}' for i in range(24)])

# Loop through each row in Demand_Unit_Data
for _, row in Demand_Unit_Data.iterrows():
    load_type = row['LoadType'].strip() # Load type (e.g., 'U_Residential', 'U_IndustryBase', 'U_IndustryPeak')
    utility_type = row['UtilityType'].strip()
    percent_load = row['% of system load'] / 100
    
    # Check if the load type exists in the Demand_Data_Normalized columns
    if load_type in season_data.columns:
        # Retrieve the hourly values for the specified load type and scale them
        hourly_values_loadtype = season_data[season_data['Hour'] <= 24][load_type].values * percent_load
        
        # Create a new row with these hourly values
        row_data = pd.DataFrame([hourly_values_loadtype], columns=[f'Hour_{i+1}' for i in range(24)])
        
        # Add identifying columns (like Load #, Zone, and LoadType) for clarity
        # row_data['Load #'] = row['Load #']
        # row_data['Zone'] = row['Zone']
        # row_data['LoadType'] = load_type
        
        # Append the new row to the result_data DataFrame
        Daily_Demand_Normalized = pd.concat([Daily_Demand_Normalized, row_data], ignore_index=True)
    # Check if the utility type exists in the Demand_Data_Normalized columns
    if utility_type in season_data.columns:
        # Retrieve the hourly values for the specified load type and scale them
        hourly_values_utilitytype = season_data[season_data['Hour'] <= 24][utility_type].values 
        
        # Create a new row with these hourly values
        row_data = pd.DataFrame([hourly_values_utilitytype], columns=[f'Hour_{i+1}' for i in range(24)])
        
        # Add identifying columns (like Load #, Zone, and LoadType) for clarity
        # row_data['Load #'] = row['Load #']
        # row_data['Zone'] = row['Zone']
        # row_data['LoadType'] = utility_type
        
        # Append the new row to the result_data DataFrame
        Daily_Demand_Utility = pd.concat([Daily_Demand_Utility, row_data], ignore_index=True)

# Reset index and inspect result
Daily_Demand_Normalized.reset_index(drop=True, inplace=True)
Daily_Demand_Utility.reset_index(drop=True, inplace=True)


#getting from normalized load to absolute load values
Daily_Demand_Absolute = pd.DataFrame(
    index=range(len(Daily_Demand_Normalized)),  # Set the number of rows
    columns=[f'Hour_{i+1}' for i in range(24)]  # Set the columns for each hour
)
for l in range(len(Daily_Demand_Normalized)):
        for h in range(len(System_Demand)):
            Daily_Demand_Absolute.iloc[l,h] = Daily_Demand_Normalized.iloc[l,h]*System_Demand['System demand (MW)'].iloc[h]





# #-----------------------------------------------------------------------------------------------------------------

#Defining the decision variables

# Defining a variable for the generators day ahead bid
Gen_DABid = model.addVars(Number_of_Generators,Number_of_Hours, vtype=GRB.CONTINUOUS, name="Gen_DABid")
    # Defining a variable for the demand day ahead bid
Dem_DABid = model.addVars(Number_of_Demand_Units,Number_of_Hours, vtype=GRB.CONTINUOUS, name="Dem_DABid")
# Define voltage angle variables for each zone
Voltage_Angle = model.addVars(Number_of_Price_Zones,Number_of_Hours, vtype=GRB.CONTINUOUS, name="Voltage_Angle")
# Define power flow variables (as before, but now dependent on voltage angles)
Power_Flow = model.addVars(len(Transmission_Capacity),Number_of_Hours, vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, name="Power_Flow")

# #-----------------------------------------------------------------------------------------------------------------

# #Defining objective function

#Objective function that maximizes social welfare
objective = (gp.quicksum(Generation_Asset_Data_Existing['C_i ($/MWh)'].iloc[g] * Gen_DABid[g,h] for g in range(Number_of_Generators) for h in range(Number_of_Hours)) 
            - gp.quicksum(Daily_Demand_Utility.iloc[d,h] * Dem_DABid[d,h] for d in range(Number_of_Demand_Units) for h in range(Number_of_Hours)))
model.setObjective(objective, GRB.MAXIMIZE)

#-----------------------------------------------------------------------------------------------------------------

#Defining Constraints

#Defining maximium Production Capacity

for p in range(Number_of_Generators):
    for h in range(Number_of_Hours):
        model.addConstr(Gen_DABid[p,h] <= Generation_Asset_Data_Existing['P_max current (MW)'].iloc[p], name=f"Production_Capacity_Constraint_{p,h}")


for tr in range(len(Transmission_Capacity)):
    for h in range(Number_of_Hours):
        print("t Number",tr)
        from_Zone = Transmission_Capacity['FromZone'][tr]  
        to_Zone = Transmission_Capacity['ToZone'][tr]  
        reactance = Transmission_Capacity['Reactance'][tr]
        
        # Flow is proportional to the voltage angle difference divided by the reactance
        model.addConstr(
            Power_Flow[tr,h] == (Voltage_Angle[from_Zone-1,h] - Voltage_Angle[to_Zone-1,h]) / reactance,
            name=f"Flow_Equation_{from_Zone}_{to_Zone}"
        )


t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 0
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1
t Number 1


C:\Users\konst\AppData\Local\Temp\ipykernel_36360\3202307770.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Daily_Demand_Normalized = pd.concat([Daily_Demand_Normalized, row_data], ignore_index=True)


In [31]:
model.write("model.lp")